In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.layers import BatchNormalization

In [2]:
(x_train,y_train), (x_test,y_test) = tf.keras.datasets.cifar100.load_data(label_mode="fine")
x_train = x_train / 255.0
x_test = x_test / 255.0
(_, y_train_sp), (_, y_test_sp) = tf.keras.datasets.cifar100.load_data(label_mode="coarse") 

In [4]:
y_train[0]

array([19])

In [3]:
class_names = [
    'apple',
    'aquarium_fish',
    'baby',
    'bear',
    'beaver',
    'bed',
    'bee',
    'beetle',
    'bicycle',
    'bottle',
    'bowl',
    'boy',
    'bridge',
    'bus',
    'butterfly',
    'camel',
    'can',
    'castle',
    'caterpillar',
    'cattle',
    'chair',
    'chimpanzee',
    'clock',
    'cloud',
    'cockroach',
    'couch',
    'crab',
    'crocodile',
    'cup',
    'dinosaur',
    'dolphin',
    'elephant',
    'flatfish',
    'forest',
    'fox',
    'girl',
    'hamster',
    'house',
    'kangaroo',
    'computer_keyboard',
    'lamp',
    'lawn_mower',
    'leopard',
    'lion',
    'lizard',
    'lobster',
    'man',
    'maple_tree',
    'motorcycle',
    'mountain',
    'mouse',
    'mushroom',
    'oak_tree',
    'orange',
    'orchid',
    'otter',
    'palm_tree',
    'pear',
    'pickup_truck',
    'pine_tree',
    'plain',
    'plate',
    'poppy',
    'porcupine',
    'possum',
    'rabbit',
    'raccoon',
    'ray',
    'road',
    'rocket',
    'rose',
    'sea',
    'seal',
    'shark',
    'shrew',
    'skunk',
    'skyscraper',
    'snail',
    'snake',
    'spider',
    'squirrel',
    'streetcar',
    'sunflower',
    'sweet_pepper',
    'table',
    'tank',
    'telephone',
    'television',
    'tiger',
    'tractor',
    'train',
    'trout',
    'tulip',
    'turtle',
    'wardrobe',
    'whale',
    'willow_tree',
    'wolf',
    'woman',
    'worm',
]
mapping_coarse_fine = {
    0: ['beaver', 'dolphin', 'otter', 'seal', 'whale'],
    1: ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
    2: ['orchid', 'poppy', 'rose', 'sunflower', 'tulip'],
    3: ['bottle', 'bowl', 'can', 'cup', 'plate'],
    4: ['apple', 'mushroom', 'orange', 'pear',
                             'sweet_pepper'],
    5: ['clock', 'computer_keyboard', 'lamp',
                                    'telephone', 'television'],
    6: ['bed', 'chair', 'couch', 'table', 'wardrobe'],
    7: ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
    8: ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
    9: ['bridge', 'castle', 'house', 'road',
                                      'skyscraper'],
    10: ['cloud', 'forest', 'mountain', 'plain',
                                     'sea'],
    11: ['camel', 'cattle', 'chimpanzee',
                                       'elephant', 'kangaroo'],
    12: ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
    13: ['crab', 'lobster', 'snail', 'spider', 'worm'],
    14: ['baby', 'boy', 'girl', 'man', 'woman'],
    15: ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
    16: ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
    17: ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree',
              'willow_tree'],
   18: ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
    19: ['lawn_mower',  'rocket', 'streetcar', 'tank', 'tractor'],
}

## Training Models for Each Superclass with Fine-Class Labels

In [5]:
superclass_models = []
histories = []  

for superclass in range(20): 
    t_indx = np.where(y_train_sp == superclass)[0]
    tst_indx = np.where(y_test_sp == superclass)[0]
    x_train_s = x_train[t_indx]
    y_train_s = y_train[t_indx]
    x_test_s = x_test[tst_indx]
    y_test_s = y_test[tst_indx]
    fine_labels_train = [mapping_coarse_fine[superclass].index(class_names[y[0]]) for y in y_train_s]
    fine_labels_test = [mapping_coarse_fine[superclass].index(class_names[y[0]]) for y in y_test_s]

    fine_labels_train = tf.keras.utils.to_categorical(fine_labels_train, 5)
    fine_labels_test = tf.keras.utils.to_categorical(fine_labels_test, 5)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='elu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='elu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='elu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), padding='same', activation='elu'))
    model.add(Flatten())
    model.add(Dense(1024, activation='elu')) 
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax')) 

    model.compile(optimizer=Adam(learning_rate=0.0001, weight_decay=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7, restore_best_weights=True)

    history = model.fit(x_train_s, fine_labels_train, validation_data=(x_test_s, fine_labels_test), epochs=30, callbacks=[early_stop])

    superclass_models.append(model)
    histories.append(history)



Epoch 1/30


79/79 [==============================] - 4s 29ms/step - loss: 1.3995 - accuracy: 0.3816 - val_loss: 1.2485 - val_accuracy: 0.4640
Epoch 2/30
79/79 [==============================] - 2s 31ms/step - loss: 1.2297 - accuracy: 0.4488 - val_loss: 1.2188 - val_accuracy: 0.4760
Epoch 3/30
79/79 [==============================] - 3s 35ms/step - loss: 1.1843 - accuracy: 0.4812 - val_loss: 1.1739 - val_accuracy: 0.4900
Epoch 4/30
79/79 [==============================] - 3s 36ms/step - loss: 1.1592 - accuracy: 0.5028 - val_loss: 1.1362 - val_accuracy: 0.5140
Epoch 5/30
79/79 [==============================] - 3s 34ms/step - loss: 1.1206 - accuracy: 0.5260 - val_loss: 1.1282 - val_accuracy: 0.5340
Epoch 6/30
79/79 [==============================] - 3s 35ms/step - loss: 1.1010 - accuracy: 0.5332 - val_loss: 1.0835 - val_accuracy: 0.5540
Epoch 7/30
79/79 [==============================] - 3s 36ms/step - loss: 1.0639 - accuracy: 0.5572 - val_loss: 1.0817 - val_accuracy: 0.5580
Epoch 8/3

In [21]:
from tensorflow.keras.models import load_model
superclass_model =load_model('model2.keras')

## Combined Prediction Function

In [22]:
def combined_predict(image):
    image = image.reshape(1,32,32,3)
    superclass_probs = superclass_model.predict(image,verbose = 0)
    superclass = np.argmax(superclass_probs)
    subclass_probs = superclass_models[superclass].predict(image,verbose = 0)
    subclass = np.argmax(subclass_probs)
    
    final_class = class_names.index(mapping_coarse_fine[superclass][subclass])
    return final_class

## Model evaluation

In [30]:
import numpy as np
correct_predictions = 0
total_predictions = len(x_test) 
for i in range(total_predictions):
    image = x_test[i]
    true_label = y_test[i][0]
    predicted_label = combined_predict(image)
    if predicted_label == true_label:
        correct_predictions += 1
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 51.63%
